In [1]:
from sys import path
from pathlib import Path
from sqlalchemy import create_engine

import pandas as pd

# Se genera la ruta absoluta al directorio "Comisiones"
path_package = str(Path.cwd().parent)

# Se agrega la ruta al directorio sys.path
path.append(path_package)

from descriptions.descriptions import descrip_sales_users_df, descrip_product_df, descrip_partner_df

# Se define la ruta para la base de datos local SQLite
db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

# Se crea el objeto engine para trabajarlo con los metodos de pandas
engine = create_engine(f'sqlite:///{db_file_path_str}')

In [2]:
file01 = 'clientes_regalos'
file01_path_str = str(Path().cwd().parent.parent.joinpath(f'data/comisiones/{file01}.xlsx'))

clientes_regalos = pd.read_excel(file01_path_str, dtype_backend='numpy_nullable')

In [4]:
clientes_regalos

,partner_id_x,partner_name_x,hotel,comprador_name,salesperson_id,salesperson_name,costo_regalo_$,fecha_regalo,comentario_regalo
0,15853,PRESTADORA DE SERVICIOS TURISTICOS Y DE GOLF,Diamante,Paulina Flores,211,Mayra Angelica Parada Manjarrez,720.0,2024-02-14,"Se compró comida, el efectivo salió de caja y ..."
1,15853,PRESTADORA DE SERVICIOS TURISTICOS Y DE GOLF,Diamante,Paulina Flores,211,Mayra Angelica Parada Manjarrez,1992.61,2024-03-12,Se regaló un Kit Compresor 20L 2 1/2Hp Pretul ...


In [ ]:
adfasdfasdfasdfasdfasdfsadfadsfasdf

In [ ]:
with engine.connect() as conn, conn.begin():

    cobranza_enero = pd.read_sql_table('cobranza_1_2024', conn, dtype_backend='numpy_nullable')
    cobranza_febrero = pd.read_sql_table('cobranza_2_2024', conn, dtype_backend='numpy_nullable')
    cobranza_marzo = pd.read_sql_table('cobranza_3_2024', conn, dtype_backend='numpy_nullable')
    cobranza_abril = pd.read_sql_table('cobranza_4_2024', conn, dtype_backend='numpy_nullable')
    cobranza_mayo = pd.read_sql_table('cobranza_5_2024', conn, dtype_backend='numpy_nullable')
    cobranza_junio = pd.read_sql_table('cobranza_6_2024', conn, dtype_backend='numpy_nullable')
    cobranza_julio = pd.read_sql_table('cobranza_7_2024', conn, dtype_backend='numpy_nullable')
    cobranza_año = pd.read_sql_table('cobranza_año_2024', conn, dtype_backend='numpy_nullable')

    costo_ventas_enero = pd.read_sql_table('costo_ventas_1_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_febrero = pd.read_sql_table('costo_ventas_2_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_marzo = pd.read_sql_table('costo_ventas_3_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_abril = pd.read_sql_table('costo_ventas_4_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_mayo = pd.read_sql_table('costo_ventas_5_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_junio = pd.read_sql_table('costo_ventas_6_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_julio = pd.read_sql_table('costo_ventas_7_2024', conn, dtype_backend='numpy_nullable')
    costo_ventas_año = pd.read_sql_table('costo_ventas_año_2024', conn, dtype_backend='numpy_nullable')

engine.dispose()